In [0]:
%sql
CREATE OR REPLACE TABLE sandbox.t_weetv.po_activation_mart
USING DELTA
PARTITIONED BY (date_ym)
AS
WITH T101 AS (
    /* KMS 관련 테이블 */
    SELECT 
      date_ym,
      mac,
      try_cast(po_date AS DATE) AS po_date,
      po_country_code,
      po_type,
      po_board_maker,
      po_odm,
      po_brand,
      po_country,
      po_year,
      po_platform
    FROM kic_data_ods.kms.kms_wee_tv
),
T102 AS (
    /* ACTIVATION: MAC별 최초 */
    SELECT
        s.mac_addr,
        s.activ_country_code,
        s.activ_date,
        s.activ_platform_code,
        s.activ_product_code,
        s.activ_sales_model
    FROM (
        SELECT
            T102.mac_addr,
            T102.Cntry_CODE   AS activ_country_code,
            DATE(T102.crt_date) AS activ_date,
            T102.Platform_code AS activ_platform_code,
            T102.Product_CODE  AS activ_product_code,
            T102.Sales_Model   AS activ_sales_model,
            ROW_NUMBER() OVER (
                PARTITION BY T102.mac_addr
                ORDER BY T102.crt_date ASC, T102.last_chg_date ASC, T102.he_etl_dt ASC
            ) AS rn
        FROM eic_data_ods.tlamp.activation_date T102
        WHERE EXISTS (
            SELECT 1
            FROM T101
            WHERE T101.mac = T102.mac_addr
        )
    ) s
    WHERE s.rn = 1
),
T103 AS (
    /* 앱 브랜드: MAC별 최초 */
    SELECT
        s.mac_addr,
        s.brand_name
    FROM (
        SELECT
            T103.mac_addr,
            T103.brand_name,
            ROW_NUMBER() OVER (
                PARTITION BY T103.mac_addr
                ORDER BY T103.src_file_date ASC, T103.he_etl_dt ASC
            ) AS rn
        FROM eic_data_ods.tlamp.apps_device_brand T103
        WHERE EXISTS (
            SELECT 1
            FROM T101
            WHERE T101.mac = T103.mac_addr
        )
    ) s
    WHERE s.rn = 1
)
SELECT
    -- 파티션 키
    T101.date_ym,

    -- PO 측 주요 컬럼
    T101.mac,
    T101.po_date,
    T101.po_country_code,
    T101.po_type,
    T101.po_board_maker,
    T101.po_odm,
    T101.po_brand,
    T101.po_country,
    T101.po_year,
    T101.po_platform,

    -- ACTIVATION 측
    T102.activ_date,
    T102.activ_country_code,
    T102.activ_platform_code,
    T102.activ_product_code,
    T102.activ_sales_model,

    -- BRAND 파싱
    SPLIT_PART(T103.brand_name, '_', 1)   AS activ_board_maker,
    SPLIT_PART(T103.brand_name, '_', 2)   AS activ_odm,
    SPLIT_PART(T103.brand_name, '_', 3)   AS activ_brand,

    -- 비교 지표
    CASE
        WHEN T101.po_country_code IS NULL THEN 'po_country_code_is_null'
        WHEN T102.activ_country_code IS NULL THEN 'not_yet_activated'
        WHEN T101.po_country_code <> T102.activ_country_code THEN 'N'
        ELSE 'Y'
    END                                      AS is_same_country,
    DATE_DIFF(T102.activ_date, T101.po_date) AS day_diff,
    
    current_timestamp() as he_etl_dt
FROM T101
LEFT JOIN T102
  ON T102.mac_addr = T101.mac
LEFT JOIN T103
  ON T103.mac_addr = T101.mac
;
